- https://dojang.io/mod/page/view.php?id=2427
- 파이썬 코딩 도장: Unit42. 데코레이터 사용하기

In [1]:
class Calc:
    @staticmethod
    def add(a, b):
        print(a+b)

- 위와 같이 많이 사용됐는데 여기서 staticmethod decorator를 사용해보자
- Closure 를 이용하여 데코레이터 생성
### function, method decorator

In [2]:
def trace(func):
    def wrapper():
        print(func.__name__, '함수 시작')
        func()
        print(func.__name__, '함수 끝')

    return wrapper

def hello():
    print('hello')

def world():
    print('world')

trace_hello = trace(hello)
trace_world = trace(world)
trace_hello()
trace_world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


- @로 decorator 사용하기

In [4]:
@trace
def hello2():
    print('hello')

@trace
def world2():
    print('world')

hello2()
world2()

hello2 함수 시작
hello
hello2 함수 끝
world2 함수 시작
world
world2 함수 끝


- parameters, return 을 처리하는 decorator

In [8]:
def trace(func):
    def wrapper(a, b):
        r = func(a, b)
        print(f'{func.__name__}(a={a}, b={b}) -> {r}')
        return r
    return wrapper

@trace
def add(a, b):
    return a+b

add(10, 20)

add(a=10, b=20) -> 30


30

- parameters가 있는 decorator

In [11]:
def is_multiple(x):
    def real_decorator(func):
        def wrapper(a, b):
            r = func(a, b)
            if r % x == 0:
                print(f'{func.__name__}의 return값은 {x}의 배수입니다.')
            else:
                print(f'{func.__name__}의 return값은 {x}의 배수가 아닙니다.')
            return r
        return wrapper
    return real_decorator

# @is_multiple(3)
# def add(a, b):
#     return a + b

@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a+b

print(add(2, 5))
print(add(10, 20))

add의 return값은 7의 배수가 아닙니다.
wrapper의 return값은 3의 배수입니다.
30
add의 return값은 7의 배수입니다.
wrapper의 return값은 3의 배수가 아닙니다.
7


- 위 add(10, 20)은 아래 순서대로 호출된다.
1. is_multiple(3)( is_multiple(7)( add(10, 20 ) )
2. is_multiple(7)을 호출하면 real_decorator()가 return
3. 그리고 이 real_decorator()에 add(10, 20)을 인자로 줌
4. 따라서 is_multiple(3)( * ) 여기서 *에 들어갈 것들을 먼저 살보면 최종적으로 real_decorator(add(10, 20))이 호출된다.
5. 여기서 wrapper(10, 20)을 실행한다.

6. 그리고 is_multiple(3)( wrapper(10, 20 ) 을 실행하므로 위와 같은 방법으로<br>
wrapper의 return값은 3의 배수입니다. 라는 출력을 한 것이다.

- 따라서 이와 같은 일을 방지하려면 @functions.wraps 라는 데코레이터를 사용해야한다.

In [12]:
from functools import wraps

def is_multiple(x):
    def real_decorator(func):
        @wraps(func)
        def wrapper(a, b):
            r = func(a, b)
            if r % x == 0:
                print(f'{func.__name__}의 return값은 {x}의 배수입니다.')
            else:
                print(f'{func.__name__}의 return값은 {x}의 배수가 아닙니다.')
            return r
        return wrapper
    return real_decorator

# @is_multiple(3)
# def add(a, b):
#     return a + b

@is_multiple(3)
@is_multiple(7)
def add(a, b):
    return a+b

print(add(2, 5))
print(add(10, 20))

add의 return값은 7의 배수입니다.
add의 return값은 3의 배수가 아닙니다.
7
add의 return값은 7의 배수가 아닙니다.
add의 return값은 3의 배수입니다.
30


## Class Decorator

- class를 활용할 때는 instance를 함수처럼 call 하는 \_\_call__ method를 구현해야 한다.

In [15]:
class Trace:
    def __init__(self, func):
        self.func = func

    def __call__(self):
        print(self.func.__name__, '함수 시작')
        self.func()
        print(self.func.__name__, '함수 끝')

@Trace
def hello():
    print('hello')

hello()

hello 함수 시작
hello
hello 함수 끝


In [18]:
def hello():
    print('hello')

trace_hello = Trace(hello)
trace_hello()

hello 함수 시작
hello
hello 함수 끝


- position arguments, keyword arguments도 처리하는 Class Decorator를 만들어본다.

In [20]:
class Trace:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        r = self.func(*args, **kwargs)
        print(f'{self.func.__name__}(args={args}, kwargs={kwargs}) -> {r}')
        return r

@ Trace
def add(a, b):
    return a + b

print(add(10, 20))
print(add(a=10, b=20))

add(args=(10, 20), kwargs={}) -> 30
30
add(args=(), kwargs={'a': 10, 'b': 20}) -> 30
30


- parameters가 있는 Class Decorator

In [22]:
class IsMultiple:
    def __init__(self, x):
        self.x = x

    def __call__(self, func):
        def wrapper(a, b):
            r = func(a, b)
            if r % self.x == 0:
                print(f'{func.__name__}의 return value는 {r}의 배수입니다.')
            else:
                print(f'{func.__name__}의 return value는 {r}의 배수가 아닙니다.')
            return r
        return wrapper

@IsMultiple(3)
def add(a, b):
    return a + b

print(add(10, 20))
print(add(2, 5))

add의 return value는 30의 배수입니다.
30
add의 return value는 7의 배수가 아닙니다.
7


- Decorator 활용 우수 사례
1. parameters 변환
2. 코드 추적 (tracing)
3. parameters validation
4. retry logic 구현
5. 반복 작업 단순화

### parameters 변환
- Design by Contract 원칙에 따라
- https://kevinx64.net/198 자세하게 설명돼있다.
- precondition, postcondition을 강제할 수도 있다.
- 보통 parameters를 형식에 맞게 변환할 때 많이 사용된다.

### code tracing
- 모니터링 하고자하는 함수의 실행과 관련한 것이다.
1. 실제 함수의 실행 경로 추적
2. 함수 지표 모니터링(CPU, memory usage등)
3. 함수의 실행 시간 측정
4. 언제 함수가 실행되고 전달된 파라미터는 무엇인지 logging

### Decorator의 부작용 처리
- Decorator function이 되기 위한 하나의 조건은 가장 안쪽에 정의된 함수여야 한다는 것이다.

In [ ]:
import logging
import time
from functools import wraps

logger = logging.getLogger(__name__)
def traced_function_wrong(function):
    logger.info(f'{function.__name__} 함수 실행')
    start_time = time.time()

    @wraps(function)
    def wrapped(*args, **kwargs):
        result = function(*args, **kwargs)
        logger.info(f'함수 {function.__name__}의 실행시간{time.time() - start_time:.2f}')
        return result
    return wrapped

@traced_function_wrong
def process_with_delay(callback, delay=0):
    time.sleep((delay))
    return callback()

여기서 마지막
'''python
@traced_function_wrong
def process_with_delay(callback, delay=0):
    time.sleep((delay))
    return callback()
'''
이 부분은 module을 import시 아래와 같이 실행된다.
'''python
process_with_delay = traced_function_wrong(process_with_wrong)
'''
따라서 start_time이 곧바로 실행되어 import 하자마자 시간이 흐르게 되는 것이다.

해결 방법은 간단하다.
start_time을 wrapping 함수 안에다 넣으면 된다.




























